## 1.1 Import Python Libraries

In [1]:
import os, sagemaker, subprocess, boto3
from datetime import datetime
from sagemaker import s3
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorchModel
from sagemaker.deserializers import JSONDeserializer

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


## 1.2 Install Ultralytics for YOLOv8 model

There are following models to choose from:
1. Object Detection:
    - yolov8l.pt
    - yolov8m.pt
    - yolov8n.pt
    - yolov8s.pt
    - yolov8x.pt
    - yolov8x6.pt
2. Segmentation:
    - yolov8l-seg.pt
    - yolov8m-seg.pt
    - yolov8n-seg.pt
    - yolov8s-seg.pt
    - yolov8x-seg.pt
3. Classification:
    - yolov8l-cls.pt
    - yolov8m-cls.pt
    - yolov8n-cls.pt
    - yolov8s-cls.pt
    - yolov8x-cls.pt
4. Pose:
    - yolov8n-pose.pt
    - yolov8s-pose.pt
    - yolov8m-pose.pt
    - yolov8l-pose.pt
    - yolov8x-pose.pt
    - yolov8x-pose-p6.pt

For further information, check the link: https://docs.ultralytics.com

In [2]:
!pip3 install ultralytics
from ultralytics import YOLO

In [12]:
!ls

1_DeployEndpoint.ipynb	2_TestEndpoint.ipynb  bus.jpg  code


In [13]:
## Choose a model:
model_name = 'code/toyotaonlybest30epochs.pt'
YOLO(model_name)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_s

In [14]:
# os.system(f'mv {model_name} code/.')

In [15]:
bashCommand = "tar -cpzf  model.tar.gz code/"
process = subprocess.Popen(bashCommand.split(), stdout=subprocess.PIPE)
output, error = process.communicate()

In [16]:
!tar -tzvf model.tar.gz


drwxrwxr-x ec2-user/ec2-user 0 2023-12-05 10:37 code/
-rw-rw-r-- ec2-user/ec2-user 1684 2023-12-05 10:37 code/inference.py
drwxrwxr-x ec2-user/ec2-user    0 2023-12-05 10:36 code/.ipynb_checkpoints/
-rw-rw-r-- ec2-user/ec2-user 1684 2023-12-05 10:37 code/.ipynb_checkpoints/inference-checkpoint.py
-rw-rw-r-- ec2-user/ec2-user   62 2023-12-05 10:26 code/requirements.txt
-rw-rw-r-- ec2-user/ec2-user 22480163 2023-12-05 10:27 code/toyotaonlybest30epochs.pt


## 1.3 Zip the code and model into `model.tar.gz` and upload it to specific S3 bucket
Here permission is granted to the S3 bucket created with CDK and not any other bucket

In [17]:
s3_client = boto3.client('s3')
bucket = "minfy-ml-bucket"
sess = sagemaker.Session(default_bucket="minfy-ml-bucket")

prefix = "yolov8/toyota-endpoint"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [19]:
sm_client = boto3.client(service_name="sagemaker")
runtime_sm_client = boto3.client(service_name="sagemaker-runtime")

account_id = boto3.client("sts").get_caller_identity()["Account"]
region = boto3.Session().region_name

role = get_execution_role()
print(f'Role: {role}')

model_data = s3.S3Uploader.upload("model.tar.gz", "s3://" + bucket + "/" + prefix)
print(f'Model Data: {model_data}')

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Role: arn:aws:iam::057834912196:role/SagemakerNotebook
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Model Data: s3://minfy-ml-bucket/yolov8/toyota-endpoint/model.tar.gz


## 1.4 Create the SageMaker PyTorchModel

In [21]:
model_name = 'toyotaonlybest30epochs.pt'

In [22]:
model = PyTorchModel(entry_point='inference.py',
                     model_data=model_data, 
                     framework_version='1.12', 
                     py_version='py38',
                     role=role,
                     env={'TS_MAX_RESPONSE_SIZE':'20000000', 'YOLOV8_MODEL': model_name},
                     sagemaker_session=sess)

## 1.5 Deploy the model on SageMaker Endpoint:

In [23]:
INSTANCE_TYPE = 'ml.m5.2xlarge'
ENDPOINT_NAME = 'yolov8-toyota-pytorch-' + str(datetime.utcnow().strftime('%Y-%m-%d-%H-%M-%S-%f'))

# Store the endpoint name in the history to be accessed by 2_TestEndpoint.ipynb notebook
%store ENDPOINT_NAME
print(f'Endpoint Name: {ENDPOINT_NAME}')

predictor = model.deploy(initial_instance_count=1, 
                         instance_type=INSTANCE_TYPE,
                         deserializer=JSONDeserializer(),
                         endpoint_name=ENDPOINT_NAME)

Stored 'ENDPOINT_NAME' (str)
Endpoint Name: yolov8-toyota-pytorch-2023-12-05-10-41-26-701524
----!